In [1]:
import pandas as pd
pd.options.display.max_columns = 200

songs = pd.read_csv('../data/spotify_clean.csv', index_col=[0])
songs = songs.drop_duplicates(['track_id'])
# Drop rows with missing values
songs.dropna(inplace=True)

In [2]:
from sklearn.preprocessing import MinMaxScaler
# Feautre transformation
# Normalize data before clustering since clustering relies on distances
scaler = MinMaxScaler()
numerical_data = songs[['popularity', 'duration_ms', 'danceability', 'energy', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']] 
scaled_numerical_data = pd.DataFrame(scaler.fit_transform(numerical_data), columns=numerical_data.columns)
# One-hot encode categorical data
categorical_col_names = ['mode', 'key', 'time_signature']
categorical_data = songs[categorical_col_names]
onehot_data = pd.get_dummies(categorical_data, columns=categorical_col_names, prefix=categorical_col_names, dtype=int)
# Merge all the features together
songs_data = pd.concat([scaled_numerical_data, onehot_data.set_axis(scaled_numerical_data.index)], axis=1)
songs_data.head()

,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,mode_0,mode_1,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,time_signature_0,time_signature_1,time_signature_3,time_signature_4,time_signature_5
0,0.73,0.042473,0.686294,0.4610,0.791392,0.148187,0.032329,0.000001,0.3580,0.718593,0.361245,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0.55,0.026971,0.426396,0.1660,0.597377,0.079067,0.927711,0.000006,0.1010,0.268342,0.318397,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0.57,0.038679,0.444670,0.3590,0.736123,0.057720,0.210843,0.000000,0.1170,0.120603,0.313643,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0.71,0.036978,0.270051,0.0596,0.573701,0.037617,0.908635,0.000071,0.1320,0.143719,0.746758,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0.82,0.036389,0.627411,0.4430,0.737103,0.054508,0.470884,0.000000,0.0829,0.167839,0.492863,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [3]:
# Convert explicit from bool to int
explicit = songs['explicit'].astype(int).to_numpy()
songs_data['explicit'] = explicit

In [4]:
from sklearn.neighbors import NearestCentroid
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

songs.reset_index(inplace=True)
genres = songs["track_genre"]
# Numerically encode the labels
label_encoder = LabelEncoder()
encoded_genres = label_encoder.fit_transform(genres)
# Using stratify might help because we have an imbalanced dataset
X_train, X_test, y_train, y_test = train_test_split(songs_data, encoded_genres, test_size=0.2, 
                                                    stratify=encoded_genres, shuffle=True, random_state=50)
# Train model
ncc = NearestCentroid(metric='euclidean')
ncc.fit(X_train, y_train)

NearestCentroid()

In [26]:
songs_data.tail()

,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,mode_0,mode_1,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,time_signature_0,time_signature_1,time_signature_3,time_signature_4,time_signature_5,explicit
89735,0.21,0.071990,0.174619,0.235,0.612952,0.043731,0.642570,0.928,0.0863,0.034070,0.517705,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
89736,0.22,0.071990,0.176650,0.117,0.577345,0.041554,0.997992,0.976,0.1050,0.035176,0.350242,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
89737,0.22,0.050276,0.638579,0.329,0.714648,0.043523,0.870482,0.000,0.0839,0.746734,0.543933,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
89738,0.41,0.052653,0.595939,0.506,0.714759,0.030777,0.382530,0.000,0.2700,0.415075,0.558651,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
89739,0.22,0.044608,0.534010,0.487,0.727429,0.075130,0.683735,0.000,0.0893,0.711558,0.325420,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [24]:
genres

0           acoustic
1           acoustic
2           acoustic
3           acoustic
4           acoustic
            ...     
89735    world-music
89736    world-music
89737    world-music
89738    world-music
89739    world-music
Name: track_genre, Length: 89740, dtype: object

In [5]:
# Evaluate model
predictions = ncc.predict(X_test)
base_accuracy = accuracy_score(y_test, predictions)
base_f1_weighted = f1_score(y_test, predictions, average='weighted')
print("Baseline performance using an Nearest Centroids")
print(f"Accuracy: {base_accuracy:.4f}")
print(f"F1-score: {base_f1_weighted:.4f}")

Baseline performance using an Nearest Centroids
Accuracy: 0.1224
F1-score: 0.0990


In [75]:
# Replace ; for spaces in the artists column
tmp = [artist.replace(";", " ") if artist else None for artist in songs['artists']]
songs['artists'] = tmp
songs['artists']

0                    Gen Hoshino
1                   Ben Woodward
2         Ingrid Michaelson ZAYN
3                   Kina Grannis
4               Chord Overstreet
                   ...          
113995             Rainy Lullaby
113996             Rainy Lullaby
113997             Cesária Evora
113998          Michael W. Smith
113999             Cesária Evora
Name: artists, Length: 89740, dtype: object

In [8]:
# Use stemming and remove stopwords
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
nltk.download('stopwords')
import re

stemmer = PorterStemmer()
# Regex for excluding numbers
token_pattern = re.compile(r"(?u)\b[a-zA-Z]+\b")
# token_pattern = re.compile(r"(?u)\b\w\w+\b")
# Probably need to add stopwords in other languages
my_stopwords = set(stopwords.words('english'))

def tokenize(text):
    stems = []
    tokens = token_pattern.findall(text)
    for item in tokens:
        if item not in my_stopwords:
            stem = stemmer.stem(item)
            if len(stem) > 2:
                stems.append(stem)
    return stems

artists_vectorizer = TfidfVectorizer(max_features=5000)
artists_matrix = artists_vectorizer.fit_transform(songs['artists'])
print(f"Number of features related to artist names: {artists_matrix.shape[1]}")
artists_df = pd.DataFrame(artists_matrix.toarray(), columns=artists_vectorizer.get_feature_names_out())
tfid_vectorizer = TfidfVectorizer(tokenizer=tokenize, max_features=5000)
concatenated_features = (songs['album_name'] + ' ' + songs['track_name']).to_list()
tfid_matrix = tfid_vectorizer.fit_transform(concatenated_features)
print(f"Number of features (removing stopwords and using stemming): {tfid_matrix.shape[1]}")
df = pd.DataFrame(tfid_matrix.toarray(), columns=tfid_vectorizer.get_feature_names_out())
songs_data.reset_index(drop=True, inplace=True)
artists_df.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
songs_data_modified = pd.concat([songs_data, df, artists_df], axis=1)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\masam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Number of features related to artist names: 5000


c:\Users\masam\anaconda3\envs\ml\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Number of features (removing stopwords and using stemming): 5000


In [9]:
songs_data_modified.describe()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,aaj,aaja,aana,aashiqui,aay,aaya,abajo,abandon,abba,abbey,abc,abcdefu,abend,abr,absenc,absolut,abstract,abuso,abyss,acaba,acapulco,access,ace,acid,acido,acoust,acoustiqu,across,act,action,activ,adagio,adam,addict,ade,adentro,adeu,adhuri,adi,adio,ador,adrenalin,adult,advanc,adventur,aesthet,affair,affect,aficionado,afraid,africa,african,afro,afrobeat,afrod,afropop,afterglow,afterlif,aftermath,afternoon,age,agent,agoni,agora,agua,ahead,ahora,aid,aida,aim,ainda,air,airplan,akatsuki,aku,akuma,akustik,ala,alabama,aladdin,alan,alarm,albert,alberto,album,...,yera,yesudas,yiruma,yo,yoasobi,yoga,yokota,yonder,yonezu,york,you,young,your,youth,yu,yui,yuki,yuna,yung,yungblud,yuri,yusei,yusuf,yuvan,yves,zabo,zac,zach,zachary,zack,zaeden,zany,zara,zarcort,zatox,zayde,zeca,zedd,zeds,zeki,zeppelin,zero,zezo,zezé,zhanna,ziegler,zillertaler,zimmer,zimmerman,zinc,zion,zoe,zombie,zondaflex,zone,zoë,zs,zschech,zuar,zubin,zucker,zuckowski,zumbi,zé,álvaro,ángeles,árbol,étienne,ícaro,ñejo,ñengo,ólafur,ötzi,şam,şanışer,şehinşah,агафонов,александр,ансамбль,большого,бунчиков,валерий,владимир,группа,джанг,добронравов,загадка,захаров,зыков,оркестр,сергей,театра,цфасман,サザンオールスターズ,ヨルシカ,吳青峰,告五人,夜光性アミューズ,櫻坂46,高爾宣
count,89740.000000,8.974000e+04,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.00000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,...,89740.000000,89740.000000,89740.00000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000
mean,33.198808,2.291444e+05,0.085848,0.56

In [64]:
# Using stratify might help because we have an imbalanced dataset
X_train, X_test, y_train, y_test = train_test_split(songs_data_modified, encoded_genres, test_size=0.2, 
                                                    stratify=encoded_genres, shuffle=True, random_state=100)
# Train model
ncc = NearestCentroid()
ncc.fit(X_train, y_train)
# Evaluate model
predictions = ncc.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
f1_weighted = f1_score(y_test, predictions, average='weighted')
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-score: {f1_weighted:.4f}")
print(f"{(f1_weighted - base_f1_weighted)/base_f1_weighted*100:.2f}% better performance when compared to baseline ncc")

Accuracy: 0.2774
F1-score: 0.2698
154.29% better performance when compared to baseline ncc
